# Performance analysis - Comparison with pybgpstream

In [ ]:
# Go to project root
import os

if "_executed_once" not in get_ipython().user_ns:
    print("This cell runs only once per kernel restart.")
    get_ipython().user_ns["_executed_once"] = True

    os.chdir("../")


This cell runs only once per kernel restart.


In [2]:
import time
import datetime
from pybgpkitstream import BGPKITStream, BGPStreamConfig
from pybgpstream import BGPStream
from tests.pybgpstream_utils import make_bgpstream


def consume_stream(stream: BGPKITStream | BGPStream):
    n_elems = 0
    start = time.time()
    for _ in stream:
        n_elems += 1
    print(f"Processed {n_elems} elements in {time.time() - start} seconds")

## No caching

Basic example

In [3]:
config = BGPStreamConfig(
    start_time=datetime.datetime(2010, 9, 1, 0, 0),
    end_time=datetime.datetime(2010, 9, 1, 2, 0),
    collectors=["route-views.sydney", "route-views.wide"],
    data_types=["updates"],
)

In [4]:
stream = BGPKITStream.from_config(config)
consume_stream(stream)


Processed 81784 elements in 11.887867212295532 seconds


In [5]:
stream = make_bgpstream(config)
consume_stream(stream)


Processed 81784 elements in 12.96371841430664 seconds


Parse two RIBs

In [6]:
config = BGPStreamConfig(
    start_time=datetime.datetime(2010, 9, 1, 0, 0),
    end_time=datetime.datetime(2010, 9, 1, 1, 59),
    collectors=["route-views.sydney", "route-views.wide"],
    data_types=["ribs"],
)

In [7]:
stream = BGPKITStream.from_config(config)
consume_stream(stream)

Processed 1819101 elements in 69.90047550201416 seconds


In [8]:
stream = make_bgpstream(config)
consume_stream(stream)


Processed 1819101 elements in 18.970858573913574 seconds


Many route collectors

In [9]:
config = BGPStreamConfig(
    start_time=datetime.datetime(2010, 9, 1, 0, 0),
    end_time=datetime.datetime(2010, 9, 1, 2, 0, 0),
    collectors=[
        "route-views.sydney",
        "route-views.wide",
        "rrc00",
        "rrc01",
        "rrc03",
        "rrc12",
    ],
    data_types=["updates"]
)

In [10]:
stream = BGPKITStream.from_config(config)
consume_stream(stream)

Processed 1142081 elements in 216.6344769001007 seconds


In [11]:
stream = make_bgpstream(config)
consume_stream(stream)

Processed 1220538 elements in 63.346800327301025 seconds


Many updates for one route collector (with [workaround fix](./many_updates.ipynb))

In [12]:
from itertools import chain, pairwise


def sample_every(
    start: datetime.datetime, end: datetime.datetime, interval: datetime.timedelta
) -> list[datetime.datetime]:
    current = start
    samples = []
    while current < end:
        samples.append(current)
        current += interval
    samples.append(end)
    return samples


def decompose_bgpstreamconfig(
    config: BGPStreamConfig, interval=datetime.timedelta(hours=2)
) -> list[BGPStreamConfig]:
    intermediate_steps = sample_every(config.start_time, config.end_time, interval)
    configs = []
    for start, end in pairwise(intermediate_steps):
        subconfig = BGPStreamConfig(
            start_time=start,
            end_time=end
            - datetime.timedelta(
                microseconds=0.1
            ),  # prevent overlapping (bgpstream includes borders)
            collectors=config.collectors,
            data_types=config.data_types,
            cache_dir=config.cache_dir,
            filters=config.filters,
        )
        configs.append(subconfig)
    return configs


config = BGPStreamConfig(
    start_time=datetime.datetime(2010, 9, 1, 0, 0),
    end_time=datetime.datetime(2010, 9, 1, 23, 55, 0),
    collectors=["route-views.wide"],
    data_types=["updates"],
)

subconfigs = decompose_bgpstreamconfig(config, interval=datetime.timedelta(hours=2))

In [13]:
stream = chain.from_iterable(
    BGPKITStream.from_config(subconfig) for subconfig in subconfigs
)
consume_stream(stream)

Processed 445076 elements in 70.42159390449524 seconds


In [14]:
stream = chain.from_iterable(make_bgpstream(subconfig) for subconfig in subconfigs)
consume_stream(stream)

Processed 445076 elements in 145.92080354690552 seconds
